In [54]:
import re

import pandas as pd

from pathlib import Path

In [104]:
runtime_root_dir = Path("runtimes/")
results_dir = Path("results/")

In [29]:
EWMA_runtimes = pd.read_csv(runtime_root_dir / "ewma_runtimes.csv", index_col=[0, 1]).reset_index(names=["Method", "Computation"])
EWMA_runtimes = (
    EWMA_runtimes.melt(var_name="Window", id_vars=["Method", "Computation"])
    .set_index(["Method", "Computation", "Window"]).unstack("Computation")
)
EWMA_runtimes.index = (
    "EWMA." + EWMA_runtimes.index.get_level_values(0).str.replace("Norm", "N").str.replace("DCS", "DCS.N").str.replace("SD", "DCS.T") 
    + "." + EWMA_runtimes.index.get_level_values(1).astype(str)
)

In [31]:
GARCH_runtimes = pd.read_csv(runtime_root_dir / "garch_runtimes.csv", index_col=[0, 1]).reset_index(names=["Method", "Computation"])
GARCH_runtimes = (
    GARCH_runtimes.melt(var_name="Window", id_vars=["Method", "Computation"])
    .set_index(["Method", "Computation", "Window"]).unstack("Computation")
)
GARCH_runtimes.index = (
    "GARCH." + GARCH_runtimes.index.get_level_values(0).str.replace("Norm", "N").str.replace("DCS", "DCS.N").str.replace("SD", "DCS.T") 
    + "." + GARCH_runtimes.index.get_level_values(1).astype(str)
)

In [50]:
GARCH_LPA_runtimes_dict = {}

for garch_lpa_runtime_file in (runtime_root_dir / "garch_lpa").glob("*.csv"):
    asset_name = garch_lpa_runtime_file.stem.split("_")[0]
    garch_lpa_asset_runtimes = pd.read_csv(garch_lpa_runtime_file)
    GARCH_LPA_runtimes_dict[asset_name] = garch_lpa_asset_runtimes.agg(["mean", "std"]).iloc[:, 0]
GARCH_LPA_runtimes = pd.DataFrame(GARCH_LPA_runtimes_dict)

In [100]:
llm_runtimes_dict = {}

for llm_runtime_file in (runtime_root_dir / "llm").glob("*.csv"):
    llm_model_name_regex = re.compile(r".*?_[a-zA-Z0-9]+_LLMTime_execution_times_(gpt[_\d\w.]+)_w(\d+)_temp0\.\d")
    llm_model_name_match = llm_model_name_regex.match(llm_runtime_file.stem)
    llm_model_name = llm_model_name_match.group(1)
    llm_window_size = int(llm_model_name_match.group(2))
    llm_runtime_data = pd.read_csv(llm_runtime_file, parse_dates=["Date"], index_col=[0])
    llm_runtimes_dict[(llm_model_name, llm_window_size)] = llm_runtime_data.agg(["mean", "std"], ).iloc[:, 0]

llm_runtimes = pd.DataFrame(llm_runtimes_dict).transpose()
llm_runtimes.index = (
    llm_runtimes.index.get_level_values(0).str.replace("gpt_3.5_turbo_instruct", "GPT-3.5").str.replace("gpt_4_turbo", "GPT-4").str.replace("gpt_4o", "GPT-4o")
    + "." + llm_runtimes.index.get_level_values(1).astype(str)
)
llm_runtimes.sort_index(inplace=True)

In [102]:
runtime_results = pd.concat([
    llm_runtimes, GARCH_LPA_runtimes.mean(axis=1).to_frame(name="GARCH.LPA").T, 
    EWMA_runtimes.droplevel(0, axis=1), GARCH_runtimes.droplevel(0, axis=1)
], axis=0)

In [105]:
runtime_results.to_latex(results_dir / "runtimes.tex", float_format="%.2f")